### Импорт библиотек

In [131]:
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from SALib.sample import saltelli
from SALib.analyze import sobol

from sklearn import model_selection, ensemble, metrics

### Считывание входных параметров

In [132]:
data_X = pd.read_csv('sample.csv')
data_X.drop('calc_id', 1, inplace = True)

data_X = data_X.iloc[:-1576]

## Заявление задачи для SALib.

В SALib анализ чувствительности Соболя происходит по следующему алгоритму: сначала задается словарь, описывающий задачу - список параметров, и границы этих параметров. Затем этот словарь используется в анализе. Сначала генерируется выборка входных параметров для модели с помощью `SALib.satelli.sample()`, а затем эта выборка должна передаться в модель. Данные из этой модели передаются в `SALib.sobol.analyze()`, который уже вычисляет индексы Соболя на основе сгенерированной выборки входных параметров и выходных данных модели.

In [126]:
problem = {
    'num_vars': 28,
    'names': list(data_X.columns),
    'bounds': [[data_X['boundary_condition_NZ'].min(), data_X['boundary_condition_NZ'].max()],
               [data_X['boundary_condition_X'].min(), data_X['boundary_condition_X'].max()],
               [data_X['boundary_condition_Y'].min(), data_X['boundary_condition_Y'].max()],
               [data_X['boundary_condition_NX'].min(), data_X['boundary_condition_NX'].max()],
               [data_X['boundary_condition_NY'].min(), data_X['boundary_condition_NY'].max()],
               [data_X['boundary_condition_Z'].min(), data_X['boundary_condition_Z'].max()],
               [data_X['load_time'].min(), data_X['load_time'].max()],
               [data_X['rock_young_constant'].min(), data_X['rock_young_constant'].max()],
               [data_X['rock_alpha_constant'].min(), data_X['rock_alpha_constant'].max()],
               [data_X['concrete_init_temp'].min(), data_X['concrete_init_temp'].max()],
               [data_X['concrete_cheat'].min(), data_X['concrete_cheat'].max()],
               [data_X['concrete_dt'].min(), data_X['concrete_dt'].max()],
               [data_X['concrete_norm_coeff'].min(), data_X['concrete_norm_coeff'].max()],
               [data_X['concrete_young_constant'].min(), data_X['concrete_young_constant'].max()],
               [data_X['concrete_alpha_constant'].min(), data_X['concrete_alpha_constant'].max()],
               [data_X['concrete_strength_time'].min(), data_X['concrete_strength_time'].max()],
               [data_X['steel_init_temp'].min(), data_X['steel_init_temp'].max()],
               [data_X['bentonite_init_temp'].min(), data_X['bentonite_init_temp'].max()],
               [data_X['bentonite_cheat'].min(), data_X['bentonite_cheat'].max()],
               [data_X['bentonite_dt'].min(), data_X['bentonite_dt'].max()],
               [data_X['bentonite_young_constant'].min(), data_X['bentonite_young_constant'].max()],
               [data_X['bentonite_alpha_constant'].min(), data_X['bentonite_alpha_constant'].max()],
               [data_X['rw_init_temp'].min(), data_X['rw_init_temp'].max()],
               [data_X['rw_cheat'].min(), data_X['rw_cheat'].max()],
               [data_X['rw_dt'].min(), data_X['rw_dt'].max()],
               [data_X['rw_young_constant'].min(), data_X['rw_young_constant'].max()],
               [data_X['rw_alpha_constant'].min(), data_X['rw_alpha_constant'].max()],
               [data_X['rw_norm_coeff'].min(), data_X['rw_norm_coeff'].max()]]
}


param_values = saltelli.sample(problem, 128)


Так как модель мы не имеем, а имеем только набор входных и выходных данных, нужно что-то придумать. Первым в голову приходит создание своей модели, которая научится на наших данных. Для этого я использовал `RandomForestRegressor`. `Cross_val_score` получился равным около 0.95, среднее квадратичное test данных от train было очень близким к нулю. Также я провел `GridSearchCV`, чтобы определить оптимальные входные параметры модели *(хотя можно вроде бы еще поизучать этот вопрос, потому что не то чтоб много разных параметров я отправил на изучение, ибо проходит поиск небыстро)*, получилось, что `n_estimators` выгоднее выбирать 75. 

In [133]:
rf_clf = ensemble.RandomForestRegressor(n_estimators = 75)

Здесь и происходит весь анализ. Поясню за некоторые моменты: `.iloc[:-1576]` здесь нужен, чтобы выкинуть лишние таргет данные, потому что `satelli.sample()` генерирует `N * (2D + 1)` параметров, где N обязательно должно быть степенью двойки (в моем случае 128), а D - количество входных признаков. Думаю, что потеря 1576 данных из 9000 не сильно сказывается на обучении, тем более, что я проверял точность обучения. Далее я обучал модель полностью `sample.csv` и `temp.csv`, а проводил предикт на сгенерированной `satelli.sample()` выборке. Таким образом, мы избавились от необходимости обращаться к физической модели, потому что обучили свою на случайном лесе. Ну и далее мы просто передаем выходные данные в `sobol.analyze()`, и получаем нужные индексы. 

Время работы следующей клетки ~10 минут, как и последующей за ней

In [128]:
%%time
points = [1,2,3,4,5,6,7,8,9,10,11,12,13]
for k in range(1,14,1):
    observe_point = k
    points.remove(observe_point)
    
    data_temp = pd.read_csv('temp.csv')
    data_temp.drop('calc_id', 1, inplace = True)
    
    for j in [5,10,20,30,40,50,150,1000]:
        for i in points:
            data_temp.drop(columns = 'temp_point'+str(i)+'_'+str(j)+'days', inplace = True)
    data_temp = data_temp.iloc[:-1576]



    sobol_indices = pd.DataFrame(index = list(data_X.columns))
    for j in [5,10,20,30,40,50,150,1000]:

        rf_clf.fit(data_X, data_temp['temp_point' + str(observe_point)+ '_' + str(j) + 'days'])
        predicted = rf_clf.predict(param_values)
        Si = sobol.analyze(problem, predicted)
        sobol_indices.insert(len(sobol_indices.columns), 'ST_'+str(observe_point)+'_'+str(j)+'days' ,Si['ST'])

    sobol_indices.to_csv('sobol_indices_temp_point' +str(k)+'.csv', float_format='%.6f')

Wall time: 7min 52s


То же самое для `stress.csv`

In [130]:
%%time
points = [1,2,3,4,5,6,7,8,9,10,11,12,13]
for k in range(1,14,1):
    observe_point = k
    points.remove(observe_point)
    
    data_stress = pd.read_csv('stress.csv')
    data_stress.drop('calc_id', 1, inplace = True)
    
    for j in [5,10,20,30,40,50,150,1000]:
        for i in points:
            data_stress.drop(columns = 'stress_point'+str(i)+'_'+str(j)+'days', inplace = True)
    data_stress = data_stress.iloc[:-1576]



    sobol_indices = pd.DataFrame(index = list(data_X.columns))
    for j in [5,10,20,30,40,50,150,1000]:

        rf_clf.fit(data_X, data_stress['stress_point' + str(observe_point)+ '_' + str(j) + 'days'])
        predicted = rf_clf.predict(param_values)
        Si = sobol.analyze(problem, predicted)
        sobol_indices.insert(len(sobol_indices.columns), 'ST_'+str(observe_point)+'_'+str(j)+'days' ,Si['ST'])

    sobol_indices.to_csv('sobol_indices_stress_point' +str(k)+'.csv', float_format='%.7f')

Wall time: 9min 11s
